<a href="https://colab.research.google.com/github/BhavyaM-23/BhavyaM-23/blob/main/AQIPrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/BhavyaM-23/sky

In [ ]:
!pip install ktrain

In [ ]:
%matplotlib inline
import os
import ktrain
from ktrain import vision as vis

In [ ]:
import re
pattern=r'([^/]+)_\d+.jpg$'
p=re.compile(pattern)
r=p.search('33_20211120131613.jpg')
print(r.group(1))

In [ ]:
DATADIR='/content/sky'

In [ ]:
(train_data,test_data,preproc)=vis.images_from_fname(DATADIR,pattern=pattern,is_regression=True,random_state=42)

In [ ]:
vis.print_image_regression_models()

In [ ]:
model=vis.image_regression_model('pretrained_mobilenet',train_data=train_data,val_data=test_data)

In [ ]:
learner=ktrain.get_learner(model=model,train_data=train_data,val_data=test_data,batch_size=64)

In [ ]:
#learning rate=1e-4

In [ ]:
learner.fit_onecycle(1e-4,5)

In [ ]:
learner.freeze(15)
learner.fit_onecycle(1e-4,3)

In [ ]:
learner.model.summary()

In [ ]:
predictor=ktrain.get_predictor(learner.model,preproc)

In [ ]:
test_data.filenames

In [ ]:
def show_prediction(fname):
  fname=DATADIR+'/'+fname
  pred=round(predictor.predict_filename(fname)[0])
  actual=int(p.search(fname).group(1))
  vis.show_image(fname)
  print("Predicted AQI: %s | Actual AQI: %s"%(pred,actual))

In [ ]:
show_prediction('33_2007002478.jpg')

In [ ]:
from tensorflow import keras

kf="m1.h5"
keras.models.save_model(learner.model,kf)

In [ ]:
import tensorflow as tf
conv=tf.lite.TFLiteConverter.from_keras_model(learner.model)
tf=conv.convert()
open("aqi.tflite","wb").write(tf)

In [ ]:
print(learner.model.predict())